# LAB: Evaluación de Modelos

## 1. Introducción

El objetivo de este LAB es realizar la evaluación de un modelos de clasificación partiendo "desde cero". Para ello, usaremos datos de transacciones inmobiliaras en Sacramento.



Importemos los paquetes requeridos:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

plt.style.use('seaborn-white')
%matplotlib inline

Carguemos el dataset de Sacramento:

In [ ]:
sac = pd.read_csv('Sacramentorealestatetransactions.csv')


Crear una variable binaria en la cual:

+ $Y = 1$ indica que la casa fue vendida por encima de 200.000 dólares
+ $Y = 0$ indica que la casa fue vendida por igual o menos de 200.000 dólares

Quedarse con las siguientes variables: `number of beds`, `baths`, `sq_ft` y el indicador de precio que generamos en el paso anterior

In [ ]:
cols = ['beds','baths','sq__ft']
X = sac[cols]


In [ ]:
y = sac['price'] > 200000

Dividan el dataset entre training y test sets. Los predictores son `beds`, `baths` y `sq ft`. El target es la variable de precio.
El tamaño del test set debe ser 33% y (opcionalmente) pueden intentar estratificar por el target.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

Fitear un modelo de regresión logística:

In [ ]:
logreg = LogisticRegression(C=1e5)

In [ ]:
logreg.fit(X_train, y_train)

In [ ]:
y_pred = logreg.predict(X_test)

Imprimir la matriz de confusión:

In [ ]:
conmat = np.array(confusion_matrix(y_test, y_pred, labels=[1,0]))

confusion = pd.DataFrame(conmat, index=['over_200k', 'under_200k'],
                         columns=['predicted_over_200k','predicted_under_200k'])

print(confusion)

Calcular la proporción de casos correctamente clasificados, la precisión y el recall.

In [ ]:
print('Accuracy  ', accuracy_score(y_test, y_pred) ,sep=' = ')
print('Recall    ', recall_score(y_test, y_pred)   ,sep=' = ')
print('Precision ', precision_score(y_test, y_pred),sep=' = ')

¿Qué dicen estas tres medidas acerca del modelo?

Supongamos que un vendedor inmboliario quiere priorizar minimizar los falsos positivos (predecir que una casa se va a vender por encima de \$200.000 cuando en realidad se vende por menos) porque este tipo de errores le hacen "perder plata".

Cambiar el umbral de decisión para **bajar la tasa de falsos positivos** e imprimir nuevamente la matriz de confusión.

In [ ]:
probs = logreg.predict_proba(X_test)
probs_1 = probs[:,1]

In [ ]:
y_pred_2 = probs_1 > 0.25

In [ ]:
confusion_matrix(y_test, y_pred_2)

In [ ]:
print('Accuracy  =', accuracy_score(y_test, y_pred_2))
print('Recall    =', recall_score(y_test, y_pred_2))
print('Precision =', precision_score(y_test, y_pred_2))

* ¿Cuál sería el lado "negativo" de bajar la tasa de falsos positivos?

Graficar la curva ROC usando las funciones provistas más arriba.

In [ ]:
fpr,tpr,_ = roc_curve(y_test, probs[:,1])

df = pd.DataFrame(dict(fpr=fpr, tpr=tpr))

plt.axis([0, 1.01, 0, 1.01])

plt.xlabel('FPR = 1 - Specificty')

plt.ylabel('TPR = Recall = Sensitivity')

plt.title('ROC Curve')

plt.plot(df['fpr'],df['tpr'])

plt.plot(np.arange(0,1, step =0.01), np.arange(0,1, step =0.01))

plt.show();

In [ ]:
auc(df['fpr'],df['tpr'])

**Bonus:** ¿Cuándo pueden ser las métricas precision y recall más útiles que la curva ROC?

Tienden a ser más útiles cuando la proporción de casos positivos es menor, dado que ambas métricas son sensibles a esto.

## BONUS: Evaluar otros modelos

Evaluar otros clasificadores además de una regresión logística para este dataset.

In [ ]:
from sklearn.svm import SVC

In [ ]:
model = SVC(C=0.01, probability=True)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
probabilidad = model.predict_proba(X_test)[:,1]
fpr,tpr,_    = roc_curve(y_test, probabilidad)
plt.plot(fpr,tpr)
plt.plot(np.arange(0,1, step =0.01), np.arange(0,1, step =0.01));

plt.axis([0, 1.01, 0, 1.01])

plt.xlabel('FPR = 1 - Specificty')
plt.ylabel('TPR = Recall = Sensitivity')


plt.title('ROC Curve');